This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# De Casteljau's Algorithm

In <cite data-cite="shoemake1985animating">Shoemake (1985)</cite>,
which famously introduces quaternions to the field of computer graphics,
Shoemake suggests to apply a variant of the
[De Casteljau's Algorithm](../euclidean/bezier-de-casteljau.ipynb)
to a quaternion control polygon,
using [Slerp](slerp.ipynb) instead of linear interpolations.

In [ ]:
def slerp(one, two, t):
    return (two * one.inverse())**t * one

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

## "Cubic"

<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>
only talks about the "cubic" case,
consisting of three nested applications of Slerp.

The resulting curve is of course not simply a polynomial of degree 3,
but something quite a bit more involved.
Therefore, we use the term "cubic" in quotes.

Shoemake doesn't talk about the "degree" of the curves at all,
they are only called "spherical Bézier curves".

In [ ]:
def de_casteljau(q0, q1, q2, q3, t):
    slerp_0_1 = slerp(q0, q1, t)
    slerp_1_2 = slerp(q1, q2, t)
    slerp_2_3 = slerp(q2, q3, t)
    return slerp(
        slerp(slerp_0_1, slerp_1_2, t),
        slerp(slerp_1_2, slerp_2_3, t),
        t,
    )

In [ ]:
q0 = angles2quat(45, 0, 0)
q1 = angles2quat(0, 0, 0)
q2 = angles2quat(-90, 90, -90)
q3 = angles2quat(-90, 0, 0)

In [ ]:
times = np.linspace(0, 1, 100)

In [ ]:
ani = animate_rotations(
    [de_casteljau(q0, q1, q2, q3, t) for t in times])

In [ ]:
display_animation(ani, default_mode='reflect')

## Arbitrary "Degree"

[splines.quaternion.DeCasteljau](../python-module/splines.quaternion.rst#splines.quaternion.DeCasteljau) class

In [ ]:
from splines.quaternion import DeCasteljau

In [ ]:
s = DeCasteljau([
    [
        angles2quat(0, 0, 0),
        angles2quat(90, 0, 0),
    ],
    [
        angles2quat(90, 0, 0),
        angles2quat(0, 0, 0),
        angles2quat(0, 90, 0),
    ],
    [
        angles2quat(0, 90, 0),
        angles2quat(0, 0, 0),
        angles2quat(-90, 0, 0),
        angles2quat(-90, 90, 0),
    ],
], grid=[0, 1, 3, 6])

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations(s.evaluate(times))

In [ ]:
display_animation(ani, default_mode='reflect')

## Constant Angular Speed

> Is there a way to construct a curve parameterized by arc length?
> This would be very useful.
>
> --<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>, section 6: "Questions"

In [ ]:
from splines import ConstantSpeedAdapter

In [ ]:
s1 = DeCasteljau([[
    angles2quat(90, 0, 0),
    angles2quat(0, -45, 90),
    angles2quat(0, 0, 0),
    angles2quat(180, 0, 180),
]])

In [ ]:
s2 = ConstantSpeedAdapter(s1)

In [ ]:
ani = animate_rotations({
    'non-constant speed': s1.evaluate(
        np.linspace(s1.grid[0], s1.grid[-1], 100)),
    'constant speed': s2.evaluate(
        np.linspace(s2.grid[0], s2.grid[-1], 100)),
})

In [ ]:
display_animation(ani, default_mode='reflect')

## Joining Curves

Until now,
we have assumed that four control quaternions are given for each segment.

If a list of quaternions is given, which is supposed to be interpolated,
the intermediate control quaternions can be computed from neighboring quaternions
as shown in
[the notebook about uniform Catmull--Rom-like quaternion splines](catmull-rom-uniform.ipynb).